In [ ]:
%pip install scikit-learn nltk numpy pandas spacy
!python -m spacy download en_core_web_sm

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import spacy
import nltk

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

## Chargement des données

In [ ]:
train = pd.read_csv('BUILD/train.csv')
dev = pd.read_csv('BUILD/dev.csv')

## Extraction des descripteurs

Le ColumnTransformer permet de calculer des descripteurs simplement à partir d'un dataframe.
Il s'agit d'une liste de 3 élements:
- le nom du descripteur (str)
- une fonction ou un objet scikit-learn qui implémente `fit` et ou `transform`
- la colonne à utiliser

Par exemple pour le descripteur `sentence_position`, on va utiliser la colonne `sentence_percent` directement (`passthrough`) (il faut utiliser des crochets pour que la taille du vecteur soit correcte).

In [ ]:
def sentence_position(df: pd.DataFrame):
    doc2nbsent = df.groupby('doc_id')['sentence_index'].max().to_dict()
    df['nb_sent'] = df['doc_id'].map(doc2nbsent.get)
    position = df['sentence_index'] / df['nb_sent']
    return position.values.reshape(-1, 1)

In [ ]:
def is_verb(tag: str):
    return tag.startswith('VB') or tag == 'MD'

def verb_tense(sentence: str):
    verbs = [tag for _, tag in nltk.pos_tag(nltk.word_tokenize(sentence)) if is_verb(tag)]
    return '' if len(verbs) == 0 else verbs[0]

In [ ]:
dev['verb_tense'] = dev['text'].map(verb_tense)
train['verb_tense'] = train['text'].map(verb_tense)

In [ ]:
verb_tense_encoding = ['VB', 'VBC', 'VBD', 'VBF', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD']

def verb_tense_encode(df: pd.DataFrame):
    return np.asarray([[1 if verb_tense_encoding[i] == vt else 0 for i in range(len(verb_tense_encoding))] for vt in df['verb_tense']])

In [ ]:
def sentence_length(df: pd.DataFrame):
    """Donne la longueur des phrases.

    Args:
        df (pd.DataFrame): Corpus
    """
    return df['text'].map(lambda x: len(x.split())).values.reshape(-1, 1)

In [ ]:
def contains_ldots(sentence: str):
    return 1 if '...' in sentence else 0

In [ ]:
dev['contains_ldots'] = dev['text'].map(contains_ldots)
train['contains_ldots'] = train['text'].map(contains_ldots)

In [ ]:
stemmer = nltk.LancasterStemmer()

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super().build_analyzer()
        return lambda doc: [stemmer.stem(w) for w in analyzer(doc)]

In [ ]:
nlp = spacy.load('en_core_web_sm')

class LemmatizedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super().build_analyzer()
        return lambda doc: [nlp(w)[0].lemma_ for w in analyzer(doc)]

In [ ]:
class PosTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super().build_analyzer()
        return lambda doc: [w + '/' + tag for w, tag in nltk.pos_tag(analyzer(doc))]

In [ ]:
classifier = make_pipeline(
    make_column_transformer(
        #(
        #    TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=10),
        #    'text'
        #),
        # ==> n'améliore pas
        #(
        #    StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=10),
        #    'text'
        #),
        (
            LemmatizedTfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=10),
            'text'
        ),
        # ==> n'améliore pas
        #(
        #    PosTfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=10),
        #    'text'
        #),
        # ==> n'améliore pas
        #(
        #    'passthrough',
        #    ['sentence_index']
        #),
        (
            'passthrough',
            ['contains_ldots']
        ),
        (
            FunctionTransformer(sentence_position),
            ['doc_id', 'sentence_index']
        ),
        # ==> n'améliore pas
        #(
        #    FunctionTransformer(lambda df: (sentence_position(df) * 100).round()),
        #    ['doc_id', 'sentence_index']
        #),
        # ==> n'améliore pas
        #(
        #    FunctionTransformer(verb_tense_encode),
        #    ['verb_tense']
        #),
        # ==> trop lent, pas d'impact significatif
        #(
        #    FunctionTransformer(sentence_length),
        #    ['text']
        #),
    ),
    LogisticRegression(multi_class='multinomial', max_iter=10000)
)

## Entraînement

In [ ]:
# On entraîne le modèle sur les 100 premiers exemples.
classifier.fit(train, train['labels'])

## Prédiction

In [ ]:
# Prédit puis choisis l'étiquette qui a la plus haut probabilité
dev['pred'] = classifier.predict(dev)
train['pred'] = classifier.predict(train)

In [ ]:
# Pour obtenir le nom des étiquettes prédites
# On récupère le nom et l'ordre des étiquettes
index2label: np.ndarray = classifier.named_steps['logisticregression'].classes_

# Retourne la probabilité pour chacune des étiquettes
predictions_prob = classifier.predict_proba(dev.iloc[:10])
# On choisis l'étiquette avec la plus haute probabilité
index_max_predictions = predictions_prob.argmax(axis=1).tolist()

# Ces deux liste devraient être les mêmes
print([index2label[l] for l in index_max_predictions])
print(classifier.predict(dev.iloc[:10]).tolist())

## Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

### Erreur d'entraînement

In [ ]:
ground_truth_labels = train['labels']
submission_labels = train['pred']

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(
    ground_truth_labels, submission_labels, average='weighted'
)

In [ ]:
# https://pyformat.info/#number
print(f'{precision:.2f}, {recall:.2f}, {f1:.2f}')

### Erreur de validation

In [ ]:
ground_truth_labels = dev['labels']
submission_labels = dev['pred']

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(
    ground_truth_labels, submission_labels, average='weighted'
)

In [ ]:
# https://pyformat.info/#number
print(f'{precision:.2f}, {recall:.2f}, {f1:.2f}')

#### Comment évoluent les scores lorsqu'on enlève un descripteur ?

| CV | SV | SP | P    | R    | f1   |
|----|----|----|------|------|------|
| x  | x  | x  | 0.63 | 0.64 | 0.62 |
| x  |    | x  | 0.63 | 0.64 | 0.61 |
| x  | x  |    | 0.54 | 0.54 | 0.52 |
| x  |    |    | 0.54 | 0.54 | 0.52 |
|    | x  | x  | 0.38 | 0.54 | 0.44 |
|    |    | x  | 0.38 | 0.53 | 0.43 |
|    | x  |    | 0.21 | 0.34 | 0.19 |

__Observations :__

* Les colonnes du *count-vectorizer* (CV) ont le plus d'impact sur le modèle.
* La colonne de *single-values* (SV) n'a qu'un très faible impact sur le modèle.

#### Comment évoluent les scores lorsqu'on change les paramètres du CountVectorizer ?

| `CountVectorizer(...)`                               | P    | R    | f1   |
|------------------------------------------------------|------|------|------|
| `stop_words='english', ngram_range=(1,3), min_df=10` | 0.63 | 0.64 | 0.62 |
| `stop_words='english', ngram_range=(1,8), min_df=10` | 0.64 | 0.64 | 0.62 |
| `stop_words='english', ngram_range=(1,3), min_df=2`  | 0.65 | 0.65 | 0.62 |
| `ngram_range=(1,3), min_df=10`                       | 0.66 | 0.66 | 0.64 |

#### Comment évoluent les scores lorsqu'on ajoute un TfidfVectorizer ?

| `...Vectorizer` | P    | R    | f1   |
|-----------------|------|------|------|
| `Count`         | 0.63 | 0.64 | 0.62 |
| `Tfidf`         | 0.68 | 0.66 | 0.62 |

### Matrice de confusion

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
confusion = confusion_matrix(ground_truth_labels, submission_labels, labels=index2label)
confusion_plot = ConfusionMatrixDisplay(confusion, display_labels=index2label)
confusion_plot.plot(xticks_rotation=60)
_